In [1]:


###############################################################################################################################################################################
import json
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from LLM import LLMPredictor
from embeddings import BGEpeftEmbedding
from langchain import FAISS
from pdfparser import extract_page_text
from bm25 import BM25Model
import torch
import jieba
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from qwen_generation_utils import make_context, decode_tokens, get_stop_words_ids
# torch.cuda.set_per_process_memory_fraction(0.93)


####对文档进行重新排序（rerank）的功能。[重点难理解]####
def rerank(docs, query, rerank_tokenizer, rerank_model, k=5):
    #将 docs 中的文档内容提取出来，存储在 docs_ 列表中。
    docs_ = []
    for item in docs:
        if isinstance(item, str):docs_.append(item)
        else:docs_.append(item.page_content)
    
    docs = list(set(docs_))
    pairs = []
    for d in docs:pairs.append([query, d])
    
    #使用 rerank_tokenizer 对查询和文档内容进行标记化，并组成查询-文档对。将标记化后的查询-文档对输入到 rerank_model 中进行评分。
    with torch.no_grad():
        inputs = rerank_tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to('cuda')
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float().cpu().tolist()
    docs = [(docs[i], scores[i]) for i in range(len(docs))]
    #将评分结果与对应的文档内容按照评分值降序排序。返回评分最高的前 k 个文档的内容。
    docs = sorted(docs, key = lambda x: x[1], reverse = True)
    docs_ = []
    for item in docs:docs_.append(item[0])
    return docs_[:k]

####用于将给定的文本 text 转换为 JSON 格式的一行字符串。####
#def create_json_line(text):
#    line_dict = {"text": text}
#    json_line = json.dumps(line_dict)
#    return json_line

####导入了 vllm 模块中的 SamplingParams 类，并创建了一个 sampling_params 对象####
#temperature：温度参数，控制生成文本的多样性。较高的温度会导致更加随机的生成结果。
#top_p：top-p 参数，用于控制采样分布的范围。它表示在累积概率达到该值时停止采样。较低的值会使得采样结果更加保守。
#max_tokens：生成文本的最大长度，限制了生成结果的长度。
from vllm import SamplingParams
sampling_params = SamplingParams(temperature=1.0, top_p=0.5, max_tokens=512) #temperature=1.0, top_p=0.5, max_tokens=512

####用于批量推断文本。【重点】####
def infer_by_batch(all_raw_text, llm, system="请根据以下给出的背景知识回答问题，对于不知道的信息，直接回答“未找到相关答案”。"):
    #首先遍历all_raw_text中的每个文本，并使用make_context函数生成推断的上下文。然后，将生成的上下文添加到 batch_raw_text 列表中。
    batch_raw_text = []
    for q in all_raw_text:
        raw_text, _ = make_context(llm.tokenizer,q,system=system,max_window_size=6144,chat_format='chatml',)
        batch_raw_text.append(raw_text)
    #接下来，调用 LLM 模型的 generate 方法对 batch_raw_text 中的文本进行生成。
    res = llm.model.generate(batch_raw_text, sampling_params, use_tqdm = False)
    res = [output.outputs[0].text.replace('<|im_end|>', '').replace('\n', '') for output in res]
    return res

def post_process(answer):
    if '抱歉' in answer or '无法回答' in answer or '无答案' in answer:
        return "无答案"
    return answer




/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-01 11:16:25,310	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
submit = False
#batch_size 变量定义了批处理大小，即一次推断的文本数量。num_input_docs 变量定义了输入文档的数量。
batch_size = 4
num_input_docs = 4
#model 变量指定了主模型的路径，如果 submit 为 True，则使用指定的提交路径，否则使用默认路径。
model = "../models/Qwen-7B-Chat" 
#embedding_path 和 embedding_path2 变量指定了嵌入模型的路径，用于文本嵌入操作。
embedding_path = "../models/gte-large-zh" 
embedding_path2 = "../models/bge-large-zh" 
#reranker_model_path 变量指定了重新排序模型的路径。
reranker_model_path = "../models/bge-reranker-large" 

llm = LLMPredictor(model_path=model, is_chatglm=False, device='cuda:0') # , temperature = 0.5; temperature=1.0, top_p=0.5
# llm.model.config.use_flash_attn = True
###############################################################################################################################################################################
#加载用于重新排序的模型和相应的分词器，并将模型设置为评估模式（eval），使用半精度浮点数（half）进行计算，并将模型移动到 GPU 上。
rerank_tokenizer = AutoTokenizer.from_pretrained(reranker_model_path)
rerank_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_path)
rerank_model.eval()  #eval() 方法将模型设置为评估模式，这意味着在推理时不会进行梯度计算。
rerank_model.half()  #half() 方法将模型参数转换为半精度浮点数，以减少内存占用和加速计算。
rerank_model.cuda()  #cuda() 方法将模型移动到 GPU 上进行加速计算。
###############################################################################################################################################################################
#    filepath = "初赛训练数据集.pdf" if not submit else "/tcdata/trainning_data.pdf"
#extract_page_text 函数从 PDF 文件中提取文本。每次提取的文本长度和重叠长度不同。提取的文本存储在 docs 列表中，其中每个元素代表一个文档对象。
#    docs = extract_page_text(filepath=filepath, max_len=300, overlap_len=100) + extract_page_text(filepath=filepath, max_len=500, overlap_len=200)
#从 docs 中提取每个文档的内容，并存储在 corpus 列表中，用于后续的处理。
#    corpus = [item.page_content for item in docs]


INFO 03-01 11:16:26 llm_engine.py:79] Initializing an LLM engine with config: model='../models/Qwen-7B-Chat', tokenizer='../models/Qwen-7B-Chat', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
WARNING 03-01 11:16:26 tokenizer.py:64] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 03-01 11:16:31 llm_engine.py:337] # GPU blocks: 378, # CPU blocks: 512
INFO 03-01 11:16:34 model_runner.py:676] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-01 11:16:34 model_runner.py:680] CUDA graphs can take additional 1~3 GiB me

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter


In [4]:
###############################################################################################################################################################################
tokenizer = AutoTokenizer.from_pretrained(model,trust_remote_code=True)

def get_token_len(text: str) -> int:

    tokens = tokenizer.encode(text)
    return len(tokens)
# 1. 从文件读取本地数据集
loader1 = TextLoader("三体1疯狂年代.txt",encoding='gbk')
documents1 = loader1.load()
loader2 = TextLoader("三体2黑暗森林.txt",encoding='gbk')
documents2 = loader2.load()
loader3 = TextLoader("三体3死神永生.txt",encoding='gbk')
documents3 = loader3.load()

documents=documents1+documents2+documents3

# 2. 拆分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=8, length_function=get_token_len,  separators = ["\n\n", "\n", "。", " ", ""])

docs = text_splitter.split_documents(documents)
corpus = [item.page_content for item in docs]
print(docs[0:2])

Token indices sequence length is longer than the specified maximum sequence length for this model (132187 > 32768). Running this sequence through the model will result in indexing errors


[Document(page_content='三体（中国科幻基石丛书） \n\u3000\u3000刘慈欣著\n\n\u3000\u3000“基石”是个平实的词，不够“炫”，却能够准确传达我们对构建中的中国科幻繁华巨厦的情感与信心，因此，我们用它来作为这套原创丛书的名字。\n\u3000\u3000最近十年，是科幻创作飞速发展的十年。王晋康、刘慈欣、何宏伟、韩松等一大批科幻作家发表了大量深受读者喜爱、极具开拓与探索价值的科幻佳作。科幻文学的龙头期刊更是从一本传统的《科幻世界》，发展壮大成为涵盖各个读者层的系列刊物。与此同时，科幻文学的市场环境也有了改善，省会级城市的大型书店里终于有了属于科幻的领地。\n\u3000\u3000仍然有人经常问及中国科幻与美国科幻的差距，但现在的答案已与十年前不同。在很多作品上 (它们不再是那种毫无文学技巧与色彩、想象力拘谨的幼稚故事)，这种比较已经变成了人家的牛排之于我们的土豆牛肉。差距是明显的——更准确地说，应该是“差别”——却已经无法再为它们排个名次。口味问题有了实际意义，这正是我们的科幻走向成熟的标志。\n\n\u3000\u3000与美国科幻的差距，实际上是市场化程度的差距。美国科幻从期刊到图书到影视再到游戏和玩具，已经形成了一条完整的产业链，动力十足；而我们的图书出版却仍然处于这样一种局面：读者的阅读需求不能满足的同时，出版者却感叹于科幻书那区区几千册的销量。结果，我们基本上只有为热爱而创作的科幻作家，鲜有为版税而创作的科幻作家。这不是有责任心的出版人所乐于看到的现状。\n\u3000\u3000科幻世界作为我国最有影响力的专业科幻出版机构，一直致力于对中国科幻的全方位推动。科幻图书出版是其中的重点之一。中国科幻需要长远眼光。需要一种务实精神，需要引入更市场化的手段，因而我们着眼于远景，而着手之处则在于一块块“基石”。\n\u3000\u3000需要特别说明的是，对于基石，我们并没有什么限定。因为，要建一座大厦需要各种各样的石料。\n\u3000\u3000对于那样一座大厦，我们满怀期待。', metadata={'source': '三体1疯狂年代.txt'}), Document(page_content='《三体》终于能与科幻朋友们见面了，用连载的方式事先谁都没有想到，也是无奈之举。之前就题材问题与编辑们仔细商讨过，

In [5]:
print(docs[-1])

page_content='“当然可以，大宇宙不会因为这五公斤就不坍缩了。”关一帆说，他还有一个没说出来的想法：也许大宇宙真的会因为相差一个原子的质量而由封闭转为开放。大自然的精巧有时超出想象，比如生命的诞生，就需要各项宇宙参数在几亿亿分之一精度上的精确配合。但程心仍然可以留下她的生态球，因为在那无数文明创造的无数小宇宙中，肯定有相当一部分不响应回归运动的号召，所以，大宇宙最终被夺走的质量至少有几亿吨，甚至可能是几亿亿亿吨。\n\u3000\u3000但愿大宇宙能够忽略这个误差。\n\u3000\u3000程心和关一帆进入了飞船，智子最后也进来了。她早就不再穿那身华丽的和服了，她现在身着迷彩服，再次成为一名轻捷精悍的战士，她的身上佩带着许多武器和生存装备，最引人注目的是那把插在背后的武士刀。\n\u3000\u3000“放心，我在，你们就在！”智子对两位人类朋友说。\n\u3000\u3000聚变发动机启动了，推进器发出幽幽的蓝光，飞船缓缓地穿过了宇宙之门。\n\u3000\u3000小宇宙中只剩下漂流瓶和生态球。漂流瓶隐没于黑暗里，在一千米见方的宇宙中，只有生态球里的小太阳发出一点光芒。在这个小小的生命世界中，几只清澈的水球在零重力环境中静静地飘浮着，有一条小鱼从一只水球中蹦出，跃入另一只水球，轻盈地穿游于绿藻之间。在一小块陆地上的草丛中，有一滴露珠从一片草叶上脱离，旋转着飘起，向太空中折射出一缕晶莹的阳光。' metadata={'source': '三体3死神永生.txt'}


In [6]:

###############################################################################################################################################################################
# embedding database创建了两个嵌入模型 BGEpeftEmbedding，并使用它们来构建两个嵌入数据库 db 和 db2，然后基于语料库 corpus 构建了一个 BM25 模型 BM25。
embedding_model = BGEpeftEmbedding(model_path=embedding_path)
db = FAISS.from_documents(docs, embedding_model)

embedding_model2 = BGEpeftEmbedding(model_path=embedding_path2)
db2 = FAISS.from_documents(docs, embedding_model2)

BM25 = BM25Model(corpus)


successful load embedding model
successful load embedding model


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.587 seconds.
Prefix dict has been built successfully.


In [7]:
###############################################################################################################################################################################
result_list = []
test_file = "santiQ.json" 
with open(test_file, 'r', encoding='utf-8') as f:
    result = json.load(f)
###############################################################################################################################################################################
#prompts[x]是控制batch推理用的，all_prompts[x]是给res3用的，来判断关键词在不在里面，ress[x]用于存结果
prompts1, prompts2, prompts3 = [], [], []
all_prompts0,all_prompts1  = [],[]
ress1, ress2, ress3 = [], [], []

for i, line in tqdm(enumerate(result)):
    # bm25 召回，使用 BM25 检索相关文档。
    search_docs1 = BM25.bm25_similarity(line['question']*3, 10)
    # bge 召回，使用嵌入数据库 db2 和 db 进行文档相似性搜索。
    search_docs2 = db2.similarity_search(line['question']*3, k=10)
    # gte 召回
    search_docs3 = db.similarity_search(line['question']*3, k=10)
    # rerank，使用重新排名模型 rerank 对检索结果进行重新排序。
    search_docs4 = rerank(search_docs1 + search_docs2 + search_docs3, line['question'], rerank_tokenizer, rerank_model, k=num_input_docs)

    #prompts[x]是控制batch推理用的，是推理的的prompts
    #prompt1用reranker召回的前4个文档合并起来作为上下文提示，prompt2直接用bm25召回的前4文档合并起来作为上下文提示生成prompt
    prompt1 = llm.get_prompt("\n".join(search_docs4[::-1]), line['question'], bm25=True)
    #prompt2 = llm.get_prompt("\n".join(search_docs1[:num_input_docs][::-1]), line['question'], bm25=True)
    prompts1.append(prompt1)
    #prompts2.append(prompt2)
    
    #all_prompts0是使用bm25召回3个结果合并作为提示，all_prompts1是使用rerank的1个结果和bm25召回2个结果合并作为提示，
    #all_prompts0.append(search_docs1[0]+'\n'+search_docs1[1]+'\n'+search_docs1[2])
    #all_prompts1.append(search_docs4[0]+'\n'+search_docs1[1]+'\n'+search_docs1[2]+search_docs1[3]+'\n'+search_docs1[4]+'\n')#
    
    if len(prompts1)==batch_size:
        ress1.extend(infer_by_batch(prompts1, llm))
        prompts1 = []
        #ress2.extend(infer_by_batch(prompts2, llm))
        #prompts2 = []
###############################################################################################################################################################################
if len(prompts1)>0:
    ress1.extend(infer_by_batch(prompts1, llm))
    #ress2.extend(infer_by_batch(prompts2, llm))



0it [00:00, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
5it [00:04,  1.19it/s]


In [8]:
###############################################################################################################################################################################
#对结果进行处理，这里的res[x]是最终的答案，这里result_list是但问题答案汇总
for i, line in enumerate(result):
    res1 = post_process(ress1[i])
    #res2 = post_process(ress2[i])
    line['answer_1'] = res1
    print(line['question'],res1,)
    result_list.append(line)
###############################################################################################################################################################################

#res_file_path = 'resSantiQ.json' 
#with open(res_file_path, 'w', encoding='utf-8') as f:
#    json.dump(result_list, f, ensure_ascii=False, indent=4)

三体组织一共建立了多少个红岸组织？ 根据材料，三体组织建立了两个红岸基地，分别位于地球和红岸基地遗址。
汪淼是做什么的？ 汪淼是一名物理学教授，是“中华二号”高能加速器项目纳米构件部分的负责人。
古筝计划的结果是什么？ 古筝行动的结果是什么？古筝行动是人类为了夺取“审判日”号上被截留的三体信息而进行的行动。但是，材料中并没有提到古筝行动的结果。
面壁计划是什么，都有哪些面壁者，他们的计划都是什么，结果如何？ 面壁计划是联合国在三体危机出现后制定的一项战略计划，由四位面壁者负责制定并执行对抗三体世界入侵的战略计划，以避开智子对人类世界无所不在的监视，从而实现战略的隐蔽性。面壁计划的核心内容是完全封闭的个人思考，制定并执行对抗三体世界入侵的战略计划。结果是面壁计划是一个完全失败的战略计划，是人类社会作为一个整体，有史以来所做出的最幼稚、最愚蠢的举动。
三体人到达地球后，对澳洲大陆做了什么事情? 三体人到达地球后，对澳洲大陆进行了殖民。
